# Introduction to Prompting

## Setup
#### Before you start these notebooks, make sure you have the OS environment variables setup. Example provided in the env.bat file for Windows in the root folder. Default variable settings for getting the OpenAI keys from your Azure Key Vault are 
- PYTHONPATH=%cd%\AzureOpenAIHelperFunctions
- KEY_VAULT_URL=https://\<YOUR AZURE KEY VAULT NAME\>.vault.azure.net/
- OPENAI_AUTH_TYPE=KeysFromAKVWithCLIAuth
#### The different options for OPENAI_AUTH_TYPE are - 
- KeysFromEnv => get the credentials from OS env
- KeysFromAKVWithCLIAuth => get the credentials from Azure Key Vault with CLI Auth
- KeysFromAKVWithMI => get the credentials from Azure Key Vault with Managed Identity
- KeysFromManagedId => authenticate with Managed Identity and get access to Azure Open AI
#### Load the API key and relevant Python libaries.

In [6]:
import openai
import sys

from azure_openai_setup import set_openai_config, get_completion, COMPLETION_MODEL_DEPLOYMENT_NAME

set_openai_config()

Got Azure OpenAI Credentials from Azure Key Vault with Azure CLI Auth


## LangChain Simple Chatbot example
The below example makes direct LangChain library calls so it is better \
for us to understand this LangChain framework.\
In subsequent examples afterwards we will create and use a helper function.

In [30]:
from langchain.chat_models import AzureChatOpenAI

# The openai.<variables> are already filled up by the above 
# set_openai_config() helper function called from above cell.
# Check that function for more details

aChat = AzureChatOpenAI(
            openai_api_key = openai.api_key,
            openai_api_base = openai.api_base,
            openai_api_version = openai.api_version,
            deployment_name = COMPLETION_MODEL_DEPLOYMENT_NAME,
            temperature=0
)

#### Asking bot to perform as task

In [31]:
from langchain import PromptTemplate, LLMChain
from langchain.prompts.chat import (
    ChatPromptTemplate,
    SystemMessagePromptTemplate,
    AIMessagePromptTemplate,
    HumanMessagePromptTemplate,
)
from langchain.schema import (
    AIMessage,
    HumanMessage,
    SystemMessage
)

aChat([HumanMessage(content="Translate this sentence from English to French. I love programming.")])

AIMessage(content="J'aime programmer.", additional_kwargs={}, example=False)

#### Generate completions for multiple sets of messaages

In [33]:
batch_messages = [
    [
        SystemMessage(content="You are a helpful assistant that translates English to French."),
        HumanMessage(content="I love programming.")
    ],
    [
        SystemMessage(content="You are a helpful assistant that translates English to French."),
        HumanMessage(content="I love artificial intelligence.")
    ],
]
result = aChat.generate(batch_messages)
for generation in result.generations:
    print(generation[0].message.content)

J'adore la programmation.
J'adore l'intelligence artificielle.


## Prompt Templates

The concept of templates make life so much easier!

#### Your Prompt Engineer creates the ChatPrompt object

and hands over the aChatPrompt to your Application Dev Engineer.

In [34]:
system_template = "You are a helpful assistance that translates to {target_language}."
aSystemPrompt = SystemMessagePromptTemplate.from_template(system_template)
human_template = "{text}"
aHumanPrompt = HumanMessagePromptTemplate.from_template(human_template)

aChatPrompt = ChatPromptTemplate.from_messages([aSystemPrompt, aHumanPrompt])

#### Your App Dev Engineer receives aChatPrompt
Populates that prompt with user input and receives completion from the LLM model

In [36]:
theCompletion = aChat(aChatPrompt.format_prompt(target_language="Pirates English", 
                                text="Hello Sir! Would you be interested in buying a bottle of rum from me, \
                                the Captain of the Black Pearl?").to_messages())

In [37]:
print(theCompletion.content)

Ahoy matey! Aye, I be interested in buying a bottle of rum from ye, the Captain of the Black Pearl. Pray tell, what be the price ye be askin' for it?


## Output Parser
Helps you get the completion parsed in your desired format. Different types of output parsers are

- PydanticOutParser
- CommaSeparatedListOutputParser
- Datetime
- Enum
- OutputFixingParser
- RetryOutputParser
- Structured Output Parser
